In [1]:
import pandas as pd


file_path = '/Users/vatsal/Desktop/consolidated_trace_data_final2.csv'
df = pd.read_csv(file_path)
df

/Users/vatsal/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


,Trace-id,HTTP Status Code,Path,User Agent
0,00000e83229182560bc00dc08d8d0895,200.0,/api/v1/nodes/08dd0fba-4f8b-4103-8a95-aed37312...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
1,0000ca9c0504724ec0352caf2d927e26,204.0,/api/v1/references/HTTPS:%2F%2FOWDS.ORG%2FAG.K...,Mozilla/5.0 (Linux; Android 14; SM-P620 Build/...
2,0000d58f79c6040f625be46853e6143f,200.0,/api/v1/references/d5731268-72f8-4350-8b42-d44...,NaN
3,0001693fd167addef160ab0dddfb31d8,200.0,/api/v1/services/8054f549-6c59-4191-afb3-d31ef...,Mozilla/5.0 (Linux; Android 10; K) AppleWebKit...
4,0001c4d4a65e06934a9e38f0f71b6496,204.0,/api/v2/nodes/ebf9e23b-3b78-4c30-9dab-cf963fb0...,Mozilla/5.0 (Linux; Android 14; SM-S911B Build...
...,...,...,...,...
186687,b411c30f385101f89919434882b3bde9,200.0,/api/v2/projects/external-id/OFCAS;ddd9d302-6a...,NaN
186688,cd4553dac4aafcc56d62819cbfab816e,200.0,/api/v2/projects/external-id/OFCAS;37171d69-4a...,NaN
186689,df940e71c54381bfa93f3194e884481c,200.0,/api/v2/projects/external-id/OFCAS;90584b67-34...,NaN
186690,ec5da04f183741db4318080fe717785e,200.0,/api/v2/projects/external-id/OFCAS;37171d69-4a...,NaN


In [3]:
# Statistical summary for numerical columns and value counts for HTTP Status Code
http_status_summary = df['HTTP Status Code'].describe()
http_status_distribution = df['HTTP Status Code'].value_counts()

http_status_summary, http_status_distribution.head()


(count    186531.000000
 mean        208.688556
 std          42.729917
 min           0.000000
 25%         200.000000
 50%         200.000000
 75%         204.000000
 max         503.000000
 Name: HTTP Status Code, dtype: float64,
 200.0    133209
 204.0     43986
 404.0      3835
 201.0      2990
 503.0      1784
 Name: HTTP Status Code, dtype: int64)

In [4]:
# Analyze the most and least frequent paths and user agents
path_distribution = df['Path'].value_counts().head(10)
user_agent_distribution = df['User Agent'].value_counts().head(10)

# Display the most frequent paths and user agents
path_distribution, user_agent_distribution


(/api/healthz                                                   36748
 /api/v2/versions                                               35871
 /api/v1/capabilities                                           10322
 /api/v1/nodes/root                                              3620
 /api/v1/oems/69bf8dd4-c21e-45ee-bb8c-a6a2d907989b               2955
 /api/v1/identities                                              2832
 /api/v2/assets/08dac0aa-8ed5-4ee8-866f-1c22990e0ef0/nodes       1209
 /api/v2/nodes/08dac0aa-8ed5-4ee8-866f-1c22990e0ef0/projects     1207
 /api/v2/nodes/c630c9a3-16de-49e6-bcfe-c34cb5032f34/projects      765
 /api/v2/assets/c630c9a3-16de-49e6-bcfe-c34cb5032f34/nodes        763
 Name: Path, dtype: int64,
 curl/8.5.0                                                                                                                                                              36748
 check_http/v2.4.0 (monitoring-plugins 2.4.0)                                                     

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import IsolationForest

# Vectorize the 'Path' column using TF-IDF for anomaly detection
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
path_tfidf = tfidf_vectorizer.fit_transform(df['Path'])

# Fit an Isolation Forest model to detect anomalies
isolation_forest = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)
df['Path_Anomaly_Score'] = isolation_forest.fit_predict(path_tfidf)

# Count flagged anomalies
anomaly_counts = df['Path_Anomaly_Score'].value_counts()

anomaly_counts


 1    185115
-1      1577
Name: Path_Anomaly_Score, dtype: int64

In [9]:
# Filter rows where Path_Anomaly_Score = -1 (anomalies)
anomalies = df[df['Path_Anomaly_Score'] == -1]



# Define suspicious patterns for SQL injection and directory traversal
sql_injection_patterns = ['UNION SELECT', 'SELECT *', 'DROP TABLE', '--', ';--', "' OR '1'='1"]
directory_traversal_patterns = ['../', '..\\', '%2e%2e%2f', '%2e%2e%5c']

# Flag suspicious paths
anomalies.loc[:, 'SQL_Injection_Flag'] = anomalies['Path'].str.contains('|'.join(sql_injection_patterns), case=False, na=False)
anomalies.loc[:, 'Directory_Traversal_Flag'] = anomalies['Path'].str.contains('|'.join(directory_traversal_patterns), case=False, na=False)


# Inspect suspicious User Agents (basic filtering for uncommon scripts)
automated_tool_patterns = ['curl', 'bot', 'spider', 'scraper']
anomalies.loc[:, 'Automated_Tool_Flag'] = anomalies['User Agent'].str.contains('|'.join(automated_tool_patterns), case=False, na=False)

# Count flagged anomalies
sql_injection_count = anomalies['SQL_Injection_Flag'].sum()
directory_traversal_count = anomalies['Directory_Traversal_Flag'].sum()
automated_tool_count = anomalies['Automated_Tool_Flag'].sum()

# Display flagged anomalies and counts
flagged_anomalies = anomalies[
    (anomalies['SQL_Injection_Flag']) | 
    (anomalies['Directory_Traversal_Flag']) | 
    (anomalies['Automated_Tool_Flag'])
]

sql_injection_count, directory_traversal_count, automated_tool_count, flagged_anomalies.head()


/Users/vatsal/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/vatsal/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


(0,
 1577,
 0,
                              Trace-id  HTTP Status Code  \
 60   0013cd6a0fc29ffedfee2edfce90ca8e             200.0   
 197  004132015c4ed208cb56173ba20dfa48             204.0   
 307  0064024be25bafb7cf449e133fa336e9             200.0   
 367  007a70bc4ecfd39896ef57e16ed51fec             204.0   
 528  00bdc626f8cc2c68b924863183ddd4da             200.0   
 
                                                   Path  \
 60   /api/v1/identities/c630c9a3-16de-49e6-bcfe-c34...   
 197  /api/v2/nodes/c630c9a3-16de-49e6-bcfe-c34cb503...   
 307  /api/v2/nodes/c630c9a3-16de-49e6-bcfe-c34cb503...   
 367  /api/v2/assets/c630c9a3-16de-49e6-bcfe-c34cb50...   
 528  /api/v2/assets/c630c9a3-16de-49e6-bcfe-c34cb50...   
 
                                             User Agent  Path_Anomaly_Score  \
 60   Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...                  -1   
 197  Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:1...                  -1   
 307  Mozilla/5.0 (Windows NT 

In [12]:
# Filter rows where the anomaly score is -1 (anomalous requests)
anomalous_requests = df[df['Path_Anomaly_Score'] == -1]

# Analyze the HTTP Status Code distribution for these anomalous rows
anomalous_http_status_distribution = anomalous_requests['HTTP Status Code'].value_counts()

# Display the distribution of HTTP Status Codes for anomalous requests
anomalous_http_status_distribution


200.0    808
204.0    769
Name: HTTP Status Code, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Assume 'log_data' is your dataset and 'label' is the target column (0=normal, 1=malicious)

# Prepare features and labels
features = df[['Path', 'User Agent', 'HTTP Status Code']]
labels = df['label']

# Vectorize text data
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
path_tfidf = tfidf_vectorizer.fit_transform(features['Path'].fillna(''))
user_agent_tfidf = tfidf_vectorizer.fit_transform(features['User Agent'].fillna(''))

# Combine features
numerical_features = features['HTTP Status Code'].fillna(0).values.reshape(-1, 1)
scaler = StandardScaler()
numerical_features_scaled = scaler.fit_transform(numerical_features)

import numpy as np
X = np.hstack((path_tfidf.toarray(), user_agent_tfidf.toarray(), numerical_features_scaled))
y = labels

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict and Evaluate
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print("AUC-ROC Score:", roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1]))


KeyError: 'label'